In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(600):
            try:
                resp = requests.get(input_.loc[a, 'Info'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp)
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
                
                if html.xpath('//span[@id="vew_partnumber[1]"]/text()')[0].strip() != input_.loc[a, 'Part_Number']:
                    continue
                    
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info']}])

                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./td[1]')[0].text.lower().strip()] = tr.xpath('./td[2]')[0].text.strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                     'Info': input_.loc[a, 'Info']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Info'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./info-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./info_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1150

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204589
[尝试次数：1] - [剩余数量：1140] - [当前时间：20:57:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204581
[尝试次数：1] - [剩余数量：1139] - [当前时间：20:57:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204593
[尝试次数：1] - [剩余数量：1138] - [当前时间：20:57:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204601
[尝试次数：1] - [剩余数量：1137] - [当前时间：20:57:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9791532
[尝试次数：1] - [剩余数量：1136] - [当前时间：20:57:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204597
[尝试次数：1] - [剩余数量：1135] - [当前时间：20:57:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9791520
[尝试次数：1] - [剩余数量：1134] - [当前时间：20:57:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9791548
[尝试次数：1] - [剩余数量：1133] - [当前时间：20:57:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204605
[尝试次数：1] - [剩余数量：1132] - [当前时间：20:57:26]

[ok

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792192
[尝试次数：2] - [剩余数量：1064] - [当前时间：20:58:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792212
[尝试次数：1] - [剩余数量：1063] - [当前时间：20:58:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792044
[尝试次数：2] - [剩余数量：1062] - [当前时间：20:58:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792224
[尝试次数：1] - [剩余数量：1061] - [当前时间：20:58:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792220
[尝试次数：1] - [剩余数量：1060] - [当前时间：20:58:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792020
[尝试次数：3] - [剩余数量：1059] - [当前时间：20:58:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792228
[尝试次数：1] - [剩余数量：1058] - [当前时间：20:58:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792216
[尝试次数：2] - [剩余数量：1057] - [当前时间：20:58:29]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792256
[尝试次数：2] - [剩余数量：1056] - [当前时间：20:58:29]

[ok] - https://www.

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792704
[尝试次数：1] - [剩余数量：987] - [当前时间：20:59:44]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792696
[尝试次数：1] - [剩余数量：986] - [当前时间：20:59:44]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792692
[尝试次数：1] - [剩余数量：985] - [当前时间：20:59:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792708
[尝试次数：1] - [剩余数量：984] - [当前时间：20:59:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792716
[尝试次数：1] - [剩余数量：983] - [当前时间：20:59:49]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792656
[尝试次数：4] - [剩余数量：982] - [当前时间：20:59:49]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792320
[尝试次数：6] - [剩余数量：981] - [当前时间：20:59:50]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792728
[尝试次数：1] - [剩余数量：980] - [当前时间：20:59:51]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9792700
[尝试次数：2] - [剩余数量：979] - [当前时间：20:59:51]

[ok] - https://www.rockauto.

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793044
[尝试次数：1] - [剩余数量：911] - [当前时间：21:00:58]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793008
[尝试次数：3] - [剩余数量：910] - [当前时间：21:01:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793064
[尝试次数：2] - [剩余数量：909] - [当前时间：21:01:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793048
[尝试次数：2] - [剩余数量：908] - [当前时间：21:01:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793076
[尝试次数：1] - [剩余数量：907] - [当前时间：21:01:03]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793028
[尝试次数：2] - [剩余数量：906] - [当前时间：21:01:03]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793068
[尝试次数：2] - [剩余数量：905] - [当前时间：21:01:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793084
[尝试次数：1] - [剩余数量：904] - [当前时间：21:01:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793056
[尝试次数：3] - [剩余数量：903] - [当前时间：21:01:06]

[ok] - https://www.rockauto.

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=15356341
[尝试次数：2] - [剩余数量：835] - [当前时间：21:01:58]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12213657
[尝试次数：2] - [剩余数量：834] - [当前时间：21:01:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793228
[尝试次数：3] - [剩余数量：833] - [当前时间：21:02:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793508
[尝试次数：1] - [剩余数量：832] - [当前时间：21:02:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793512
[尝试次数：1] - [剩余数量：831] - [当前时间：21:02:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793472
[尝试次数：3] - [剩余数量：830] - [当前时间：21:02:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793500
[尝试次数：1] - [剩余数量：829] - [当前时间：21:02:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=11342737
[尝试次数：3] - [剩余数量：828] - [当前时间：21:02:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9793516
[尝试次数：1] - [剩余数量：827] - [当前时间：21:02:03]

[ok] - https://www.rockau

...

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9795892
[尝试次数：3] - [剩余数量：303] - [当前时间：21:09:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9795900
[尝试次数：3] - [剩余数量：302] - [当前时间：21:09:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9795908
[尝试次数：2] - [剩余数量：301] - [当前时间：21:09:17]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12677817
[尝试次数：1] - [剩余数量：300] - [当前时间：21:09:18]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9795416
[尝试次数：4] - [剩余数量：299] - [当前时间：21:09:18]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9795952
[尝试次数：1] - [剩余数量：298] - [当前时间：21:09:18]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9795940
[尝试次数：1] - [剩余数量：297] - [当前时间：21:09:18]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9795948
[尝试次数：1] - [剩余数量：296] - [当前时间：21:09:18]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9795936
[尝试次数：3] - [剩余数量：295] - [当前时间：21:09:19]

[ok] - https://www.rockauto

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796104
[尝试次数：1] - [剩余数量：227] - [当前时间：21:10:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796112
[尝试次数：1] - [剩余数量：226] - [当前时间：21:10:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204977
[尝试次数：1] - [剩余数量：225] - [当前时间：21:10:46]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204981
[尝试次数：1] - [剩余数量：224] - [当前时间：21:10:47]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204993
[尝试次数：1] - [剩余数量：223] - [当前时间：21:10:48]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204973
[尝试次数：3] - [剩余数量：222] - [当前时间：21:10:48]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205005
[尝试次数：1] - [剩余数量：221] - [当前时间：21:10:49]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12204933
[尝试次数：2] - [剩余数量：220] - [当前时间：21:10:50]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796096
[尝试次数：2] - [剩余数量：219] - [当前时间：21:10:53]

[ok] - https://www.roc

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12208597
[尝试次数：1] - [剩余数量：151] - [当前时间：21:12:03]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12208593
[尝试次数：1] - [剩余数量：150] - [当前时间：21:12:03]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12208601
[尝试次数：2] - [剩余数量：149] - [当前时间：21:12:07]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12208609
[尝试次数：1] - [剩余数量：148] - [当前时间：21:12:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205181
[尝试次数：3] - [剩余数量：147] - [当前时间：21:12:10]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796184
[尝试次数：4] - [剩余数量：146] - [当前时间：21:12:10]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796192
[尝试次数：1] - [剩余数量：145] - [当前时间：21:12:11]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205153
[尝试次数：3] - [剩余数量：144] - [当前时间：21:12:11]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12208573
[尝试次数：2] - [剩余数量：143] - [当前时间：21:12:11]

[ok] - https://www.ro

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205437
[尝试次数：2] - [剩余数量：75] - [当前时间：21:13:07]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205313
[尝试次数：4] - [剩余数量：74] - [当前时间：21:13:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205441
[尝试次数：2] - [剩余数量：73] - [当前时间：21:13:10]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205337
[尝试次数：2] - [剩余数量：72] - [当前时间：21:13:10]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796260
[尝试次数：1] - [剩余数量：71] - [当前时间：21:13:10]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12208649
[尝试次数：2] - [剩余数量：70] - [当前时间：21:13:11]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796264
[尝试次数：1] - [剩余数量：69] - [当前时间：21:13:12]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12208653
[尝试次数：1] - [剩余数量：68] - [当前时间：21:13:12]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205445
[尝试次数：1] - [剩余数量：67] - [当前时间：21:13:13]

[ok] - https://www.rockauto.co

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796320
[尝试次数：1] - [剩余数量：0] - [当前时间：21:14:12]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796544
[尝试次数：1] - [剩余数量：0] - [当前时间：21:14:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205617
[尝试次数：4] - [剩余数量：0] - [当前时间：21:14:14]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796516
[尝试次数：3] - [剩余数量：0] - [当前时间：21:14:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12208685
[尝试次数：2] - [剩余数量：0] - [当前时间：21:14:24]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=12205541
[尝试次数：4] - [剩余数量：0] - [当前时间：21:14:24]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796540
[尝试次数：7] - [剩余数量：0] - [当前时间：21:15:08]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=6628&pk=9796324
[尝试次数：5] - [剩余数量：0] - [当前时间：21:15:18]

输出ing...

Done ~
